In [35]:
import pandas as pd
import re

# Modalities A and B
The A and B modalities are very similar in the way they're structured, so it makes sense to treat them similarly. For the Modality C, some few adjustments are going to have to be made, so we'll deal with it later.<br><br>

<b>Objectives</b>
- Retrieve the amount of right answers per question

In [360]:
# Reading individual rooms results
modalities = {
    'A': [
        open('./data/mod_a_sala_4.txt', encoding='utf-8').read(),
        open('./data/mod_a_sala_5.txt', encoding='utf-8').read(),
        open('./data/mod_a_sala_6.txt', encoding='utf-8').read(),
        open('./data/mod_a_sala_7.txt', encoding='utf-8').read()
    ],
    'B': [
        open('./data/mod_b_sala_11.txt', encoding='utf-8').read(),
        open('./data/mod_b_sala_12.txt', encoding='utf-8').read(),
        open('./data/mod_b_sala_13.txt', encoding='utf-8').read(),
        open('./data/mod_b_sala_14.txt', encoding='utf-8').read(),
        open('./data/mod_b_sala_15.txt', encoding='utf-8').read(),
        open('./data/mod_b_extra.txt', encoding='utf-8').read()
    ]
}

In [371]:
# Using regex to retrieve only the answer list

answers = {'A': [], 'B': []}
for modality in modalities:
    for room in modalities[modality]:
        
        # Removing break lines and whitespaces for easier regex pattern
        string = room.replace('\n', '').replace(' ','');

        # .*? - Anything until you find the next pattern
        # \d* - Any combination of digits (e.g.: 1, 5, 20), indicating the question number
        # \.  - The dot character between the question number and the option marked
        # \w  - Any character, indicating the option that the student marked
        # \+* - The plus signal, indicating if it's the correct answer (asterisk because it might not be a right answer)
        matches = re.findall('.*?(\d*\.\w\+*)', string)

        # extend() instead of append() in order to add each individual element to list
        answers[modality].extend(matches)

In [380]:
# Checking which answers are right ones based on weather or not  it contains a plus sign besides it

right_answers = {'A': [], 'B': []}
for modality in answers:
    for answer in answers[modality]:
        
        # Matches two groups: the question number and the plus sign
        matches = re.match('(\d*)\.\w(\+)*', answer)
        number = matches.group(1)
        is_right = matches.group(2)

        # Checking if answer list is over and it's a right question
        if (number and is_right):
            question_number = int(number)

            # Appends 0 to the list if it hasn't already
            if (len(right_answers[modality]) < question_number): right_answers[modality].append(1)
            else: right_answers[modality][question_number-1] += 1